# An introduction to profiling in python

Let's take a look at some benchmarking and profiling tools.

## The time module

Starting with doing it yourself using the time module.

`time.perf_counter_ns()` is the maximum resolution time function
returning values in nano-seconds (because floating point seconds
will sometimes not have the resolution to represent short functions).

We'll time a simple MC calculation of pi.


In [1]:
import time
import python_pi

In [2]:
for n in [1000, 10000, 100000, 1000000, 10000000]:
    start = time.perf_counter_ns()
    pi = python_pi.pi_python(n)
    end = time.perf_counter_ns()
    t = end - start
    print(f"{n:10} samples pi is {pi:7f} in {t/1E9:7f} s ({t/n:7f} ns/iteration)")

      1000 samples pi is 3.123123 in 0.000400 s (399.901000 ns/iteration)
     10000 samples pi is 3.139514 in 0.003895 s (389.526800 ns/iteration)
    100000 samples pi is 3.143031 in 0.037892 s (378.920130 ns/iteration)
   1000000 samples pi is 3.140991 in 0.365317 s (365.317259 ns/iteration)
  10000000 samples pi is 3.141678 in 3.509860 s (350.985974 ns/iteration)


## Timeit

We also have `timeit`, a python module designed to give us this kind of information.

It's built into ipython so you can use the 'magic' (%) syntax or you can import it as a module
or run it from the command line.

Remember that
1 s = 1000 ms = 1,000,000 $\mu$s = 1,000,000,000 ns

In [3]:
%timeit python_pi.pi_python(1000)
# Timing stats for the line

349 µs ± 6.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%%timeit

python_pi.pi_python(1000)
# Timing stats for the cell

343 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
import timeit

t = timeit.Timer('python_pi.pi_python(1000)',setup="from __main__ import python_pi")
tottime = t.timeit(number=10000)
time_per_loop_ms = tottime/10000 * 1.0E6
print(f"1000 samples took {time_per_loop_ms} microseconds")

1000 samples took 318.7780234 microseconds


## cProfile

That all tells us how long something takes. Very useful if you are trying to make something faster (are you done yet) but what about finding out where to spend effort? For this we need a proper profiler.

In general profiling comes in two types **deterministic**, where function calls are instrumented and the time spent in each function computed, and **statistical** where the processor reports what it's doing every so often and this is converted into the amount of time spent on each line of code. Being an interpreted language in python we can use a deterministic approach. The same is true of the profiler built into the Matlab environment.

So we'll run our MC code under a profiler. Again we will use the ipython magic but you can do this on the command line too

In [6]:
%%prun -s cumulative
python_pi.pi_python(10000)

         20008 function calls in 0.007 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.007    0.007 {built-in method builtins.exec}
        1    0.000    0.000    0.007    0.007 <string>:1(<module>)
        1    0.005    0.005    0.007    0.007 python_pi.py:4(pi_python)
    20000    0.002    0.000    0.002    0.000 {method 'random' of '_random.Random' objects}
        1    0.000    0.000    0.000    0.000 random.py:126(seed)
        1    0.000    0.000    0.000    0.000 {function Random.seed at 0x108f498b0}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

## Line_profiler

The profiler helps find which function to look at, but how do we find out which line is slow.
This is where a line profiler helps. In python this is also deterministic but you need to install a package. Run
`conda install line_profiler` in your environment to do this.

You also need to specify which function(s) to profile. So do this after you have found interesting functions.

In [7]:
%load_ext line_profiler

In [8]:
%lprun -f python_pi.pi_python python_pi.pi_python(10000)

Timer unit: 1e-06 s

Total time: 0.02821 s
File: /Users/andreww/Code/CompEarthSci/profile_example/python_pi.py
Function: pi_python at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def pi_python(num_samps, prn_seed=101):
     5                                               """
     6                                               Monte Carlo method of calculating pi
     7                                             
     8                                               num_samps is the number of samples in the unit square
     9                                               to probe.
    10                                             
    11                                               """
    12         1         28.0     28.0      0.1      random.seed(prn_seed)
    13         1          1.0      1.0      0.0      pi = 0
    14         1          0.0      0.0      0.0      n = 0
    15     10001       4234.0     

## How to make it faster?